In [14]:
# You'll have to install NPZ and PyPlot (PyPlot is a lot better than native julia plotting and has same syntax as matplotlib from python)
using LinearAlgebra, NPZ, Random, PyPlot, Random, Distributions


# Theta is an N-dimensional vector. J is NxN matrix.
function motionalEnergy(θ,J)
    E = 0
    @inbounds for i=1:length(θ)
        @inbounds for j=i:length(θ)
            if i==j
                E += -J[i,j]*cos(θ[i]+θ[j])
            else
                E += -2*J[i,j]*cos(θ[i]+θ[j])
            end
        end
    end
    return E
end

# Theta is again an N-dimensional vector. Tfunc is a function that accepts a step (int or float)
# and returns a temperature (float). Steps is the number of metropolis steps to do. It should be a large number
# scale proportional to N.
@views function Metropolis_mo!(θ, Tfunc, J, steps; stepSize=π/8, stepOffset=0)

    N = length(θ)

    # Do a bunch of Metropolis steps
    @inbounds for step=1:steps

        idx = rand(1:N)
        dθ = randn()*stepSize
        dE = 0
        T = Tfunc(step+stepOffset)
        @inbounds for k=1:N
            if k==idx
                dE -= J[idx,idx]*( cos(2*(θ[idx]+dθ)) - cos(2*θ[idx]) )
            else
                dE -= 2*J[k,idx]*( cos(θ[k]+θ[idx]+dθ) - cos(θ[k]+θ[idx]) )
            end
        end

        # See if the random number is less than the Boltzmann weight
        if rand() < exp(-dE/T)
            θ[idx] += dθ
        end
        
    end
end

# This computes the 2x2 overlap tensor for two replicas.
function olapTensor(θ1,θ2)
   
    N = length(θ1)
    x1 = cos.(θ1)
    y1 = sin.(θ1)
    
    x2 = cos.(θ2)
    y2 = sin.(θ2)
    
    olap = zeros(2,2)
    
    olap[1,1] = dot(x1,x2)/N
    olap[1,2] = dot(x1,y2)/N
    olap[2,1] = dot(y1,x2)/N
    olap[2,2] = dot(y1,y2)/N
    
    return olap
end

# This is the nonlocal interaction in a confocal cavity.
# ϕ>0 models imperfect degeneracy. Sx and Sy are the size of Gaussian atomic distributions
# in units of the fundamental mode waist w00.
function Dnon(x1,x2,y1,y2,ϕ,sx,sy)

    wx2 = 1 + 2*sx^2
    wy2 = 1 + 2*sy^2

    ϕx = log((1+2*sx^2)/(1-2*sx^2))
    ϕy = log((1+2*sy^2)/(1-2*sy^2))

    prefactor = 4/(π * wx2 * wy2 * sqrt((1+exp(-2*(ϕ+ϕx)))*(1+exp(-2*(ϕ+ϕy)))) )

    arg  = -(1-exp(-ϕ)/(wx2*cosh(ϕ+ϕx))) * (x1^2+x2^2)/wx2
    arg += -(1-exp(-ϕ)/(wy2*cosh(ϕ+ϕy))) * (y1^2+y2^2)/wy2

    cosArg = 2*exp(ϕx)*x1*x2 / (wx2^2*cosh(ϕ+ϕx)) + 2*exp(ϕy)*y1*y2 / (wy2^2*cosh(ϕ+ϕy))

    return prefactor * (exp(arg+im*cosArg) + exp(arg-im*cosArg))/2
end

# Makes J matrices using confocal connectivity. 
function confocalJreal(N;ϕ=0,s=0.1,w=2,pos=-1,precision=3)

    if pos==-1
        pos = randn(N,2)*w
    end
    x = pos[:,1]
    y = pos[:,2]

    J = zeros(N,N)
    for i=1:N
        for j=i:N
            J[i,j] = Dnon(x[i],x[j],y[i],y[j],ϕ,s,s)
            J[j,i] = J[i,j]
        end
    end

    return pos,J
end

confocalJreal (generic function with 1 method)

In [20]:
function isingEnergy(S,J)
    E = 0
    @inbounds for i=1:length(S)
        @inbounds for j=(i+1):length(S)
            if i==j
                E += -J[i,j]*S[i]*S[j]
            else
                E += -2*J[i,j]*S[i]*S[j]
            end
        end
    end
    return E
end

function gradientDescent(J, S)
    N = size(J)[1]
    foundMinimum = false
    while !foundMinimum
        Sstart = copy(S)
        # Choose a spin to flip
        bestIdx = 0
        mindE = 0
        @inbounds for idx=1:N
            dE = 0
            @inbounds for k=1:N
                if k==idx
                    dE -= 0
                else
                    dE += 4*J[k,idx]*(S[k]*S[idx])
                end
            end
            
            if mindE > dE
                bestIdx = idx
                mindE = dE
            end
        end
        println("Energy: ", isingEnergy(S,J))
        println(mindE, " ", bestIdx, " ", S)
        # if no spin flip lower energy, we are at a minimum
        if mindE > -1e-9
            foundMinimum = true
        else
            S[bestIdx] = -S[bestIdx]
        end
    end
    return S
end

N = 4
S = rand((-1,1),N)
d = Normal(0.0, 1.0)
J = rand(d,(N,N))
println(S)
print(gradientDescent(J, S))

[-1, 1, -1, 1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [

-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.

-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534

-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3

-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8

...

-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8

-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534

Excessive output truncated after 524292 bytes.

1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3 [-1, 1, 1, -1]
Energy: 4.712343512110996
-8.153496741831958 4 [-1, 1, -1, -1]
Energy: -3.441153229720962
-8.319563822082046 1 [-1, 1, -1, 1]
Energy: -11.251125781821315
-1.6651122243599947 3 [1, 1, -1, 1]
Energy: 0.7048642456534795
-7.845435278787423 1 [1, 1, 1, 1]
Energy: 4.405342666683167
-9.77839773585989 4 [-1, 1, 1, 1]
Energy: -5.373055069176722
-3.5357036705470706 3

LoadError: InterruptException: